In [2]:
from offset_matrix import OffsetTensor
import imageio.v3 as iio
import numpy as np
import itertools
import fast1dkmeans

from wavelet import Wavelet
from periodic.wave import wavedec_period

In [3]:
data = OffsetTensor(iio.imread('test/lenna.bmp'), np.array([0, 0]))
#data = OffsetMatrix(28. * np.array([[1, 2, 3, 4, 5, 6], [2, 3, 4, 5, 6, 7], [3, 4, 5, 6, 7, 8], [4, 5, 6, 7, 8, 9], [5, 6, 7, 8, 9, 10], [6, 7, 8, 9, 10, 11]]), np.array([0,0]))

M = np.array([[


1, -1], [1,1]])

h = OffsetTensor(np.array([[0.25, 0.5, 0.25]]), np.array([0, -1]))
g = (OffsetTensor(np.array([[-0.125, -0.25, 0.75, -0.25, -0.125]]), np.array([0, -1])),)
hdual = OffsetTensor(np.array([[-0.125, 0.25, 0.75, 0.25, -0.125]]), np.array([0, -2]))
gdual = (OffsetTensor(np.array([[-0.25, 0.5, -0.25]]), np.array([0, 0])),)



w = Wavelet(h, g, hdual, gdual, M, np.abs(np.linalg.det(M)))

#ci_ = wavedec(data, 1, w)
ci = wavedec_period(data, w, 2)
#clamp(ci)


In [4]:
n_cluster = 256
flat_coef = np.array(list(ci[0]) + list(itertools.chain(*itertools.chain(*ci[1:]))))
clusters = fast1dkmeans.cluster(flat_coef, n_cluster, method='binary-search-interpolation')

In [5]:
labels, inverse = np.unique(clusters, return_inverse=True)

In [8]:
sums = np.bincount(inverse, weights=flat_coef)

In [10]:
counts = np.bincount(inverse)

In [13]:
centroids = sums / counts

In [14]:
clusters

array([185, 220, 201, ...,  36,  32,   4], shape=(262144,), dtype=int32)

In [16]:
flat_restored = centroids[clusters]

In [23]:
restored = list()
idx = 0
restored.append(flat_restored[:ci[0].size])
idx = ci[0].size

In [24]:
for c in ci[1:]:
    restored.append([])
    for cc in c:
        restored[-1].append(flat_restored[idx:idx+cc.size])
        idx += cc.size

In [25]:
restored

[array([190.28457084, 217.98145196, 203.63271467, ..., 150.41427239,
        166.34791368, 188.51964286], shape=(65536,)),
 [array([-42.34375   , -16.14714404, -36.18005952, ...,  -2.28221264,
          -0.99847816,   9.07416667], shape=(65536,))],
 [array([-9.98478157e-01, -9.98478157e-01, -1.77375412e-06, ...,
         -1.77375412e-06, -9.98478157e-01, -2.56939655e+01], shape=(131072,))]]